In [3]:
import numpy as np
import itertools

N = 10 ## Cambiar lectura de matriz
k = N-1
covmat = np.random.rand(N,N) #Aca corresponde ingresar la matriz de covarianzas ya calculada
covmat = 0.5 * (covmat + covmat.T)
covmat[[range(0,N)],[range(0,N)]] = 1 #Esta linea esta demas, eliminar cuando se vea como recibir la matriz desde matlab
arr = list(range(0,N)) #Arreglo para generar las mascaras de las submatrices cuadradas

In [4]:
covmat

array([[1.        , 0.72801341, 0.43107376, 0.59516421, 0.48272556,
        0.89878844, 0.28636567, 0.80243102, 0.54899989, 0.46707298],
       [0.72801341, 1.        , 0.404122  , 0.27481531, 0.29433314,
        0.5709858 , 0.55709283, 0.79518523, 0.72934671, 0.10568877],
       [0.43107376, 0.404122  , 1.        , 0.80947645, 0.37877564,
        0.1195931 , 0.37037753, 0.66511973, 0.70241825, 0.60503463],
       [0.59516421, 0.27481531, 0.80947645, 1.        , 0.82845422,
        0.34938354, 0.4582764 , 0.2977485 , 0.21652971, 0.09750683],
       [0.48272556, 0.29433314, 0.37877564, 0.82845422, 1.        ,
        0.70149107, 0.53558233, 0.54715048, 0.36213859, 0.52255734],
       [0.89878844, 0.5709858 , 0.1195931 , 0.34938354, 0.70149107,
        1.        , 0.52239201, 0.48553739, 0.41746876, 0.19889873],
       [0.28636567, 0.55709283, 0.37037753, 0.4582764 , 0.53558233,
        0.52239201, 1.        , 0.3757081 , 0.36323384, 0.29428651],
       [0.80243102, 0.79518523, 0.6651197

In [5]:
from ipyparallel import Client
rc = Client(profile='brincolab-cluster')
rc.purge_everything()  #Limpiar la sesion

/opt/miniconda/envs/jupyterhub/lib/python3.6/site-packages/ipyparallel/client/client.py:459: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@brincolab1')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


In [6]:
dview = rc[:] # use all engines

In [7]:
dview.use_pickle()

<AsyncResult: use_pickle>

In [8]:
dview['covmat'] = covmat #Distribuimos la matriz de covarianza

In [9]:
c = dview['covmat']

In [10]:
c

[array([[1.        , 0.72801341, 0.43107376, 0.59516421, 0.48272556,
         0.89878844, 0.28636567, 0.80243102, 0.54899989, 0.46707298],
        [0.72801341, 1.        , 0.404122  , 0.27481531, 0.29433314,
         0.5709858 , 0.55709283, 0.79518523, 0.72934671, 0.10568877],
        [0.43107376, 0.404122  , 1.        , 0.80947645, 0.37877564,
         0.1195931 , 0.37037753, 0.66511973, 0.70241825, 0.60503463],
        [0.59516421, 0.27481531, 0.80947645, 1.        , 0.82845422,
         0.34938354, 0.4582764 , 0.2977485 , 0.21652971, 0.09750683],
        [0.48272556, 0.29433314, 0.37877564, 0.82845422, 1.        ,
         0.70149107, 0.53558233, 0.54715048, 0.36213859, 0.52255734],
        [0.89878844, 0.5709858 , 0.1195931 , 0.34938354, 0.70149107,
         1.        , 0.52239201, 0.48553739, 0.41746876, 0.19889873],
        [0.28636567, 0.55709283, 0.37037753, 0.4582764 , 0.53558233,
         0.52239201, 1.        , 0.3757081 , 0.36323384, 0.29428651],
        [0.80243102, 0.7951

In [11]:
dview.block = True #Modo block

In [12]:
from math import factorial, ceil
from itertools import islice, chain

def batch(iterable, size):
    sourceiter = iter(iterable)
    while True:
        batchiter = islice(sourceiter, size)
        yield chain([next(batchiter)], batchiter)

iteraciones = {}
det_calcular = {}
indices_scatter = {}
for i in range(2,N):
    iteraciones[i] = itertools.combinations(arr, i)
    iters = itertools.combinations(arr, i)
    n = int(factorial(N) / factorial(i) / factorial(N-i))
    n_per_process = ceil(int(n)/len(rc.ids) )
    print(i, n, n_per_process)
    det_calcular[i] = (n,n_per_process)
    indexes = []
    for j in range(0,n,n_per_process):
        sl = islice(iters, j , j + n_per_process )
        indexes.append(sl)
        
    indices_scatter[i] = indexes

2 45 1
3 120 2
4 210 4
5 252 4
6 210 4
7 120 2
8 45 1
9 10 1


In [13]:
iteraciones

{2: <itertools.combinations at 0x7fb2d4698db8>,
 3: <itertools.combinations at 0x7fb2d45af638>,
 4: <itertools.combinations at 0x7fb2d45b19f8>,
 5: <itertools.combinations at 0x7fb2d45b2b88>,
 6: <itertools.combinations at 0x7fb2d453d098>,
 7: <itertools.combinations at 0x7fb2d453e228>,
 8: <itertools.combinations at 0x7fb2d45395e8>,
 9: <itertools.combinations at 0x7fb2d453f4f8>}

In [14]:
with dview.sync_imports():
    import numpy

importing numpy on engine(s)


In [15]:
def det(N):
    dets = numpy.zeros(N,dtype=numpy.float32)
    if len(index_mat) > 0:
        idx_mat = index_mat[0]
    else:    
        return {'dets':[],'matrices':[],'idx':[]}
    matrices = []
    idxs = []
    i=0
    for ind in idx_mat:
        idx = list(ind)
        matrix = covmat[idx][:,idx]
        dets[i] = numpy.linalg.det(matrix)
        matrices.append(matrix)
        idxs.append(idx)
        i += 1
    return {'dets':dets,'idx':idxs}

In [16]:
def entropy(x,y):
    return 0.5*np.log((2*np.pi * np.exp(1))**(x)*y)

In [17]:
entropy(2,3)

3.3871832107434003

In [18]:
responses = {}
for i in range(2,N):
    n_dets = det_calcular[i][0]
    n_scatters = det_calcular[i][1]
    print(n_scatters)
    dview.scatter('index_mat',indices_scatter[i])
    ar = dview.apply_async(det,n_scatters)
    ar.wait()
    responses[i] = ar.get()

1
2
4
4
4
2
1
1


In [ ]:
ar.metadata

In [20]:
#for response in responses:
#    print(len(responses[response]),responses[response][0].shape)

responses[2]

[{'dets': array([0.46999648], dtype=float32), 'idx': [[0, 1]]},
 {'dets': array([0.8141754], dtype=float32), 'idx': [[0, 2]]},
 {'dets': array([0.64577955], dtype=float32), 'idx': [[0, 3]]},
 {'dets': array([0.76697606], dtype=float32), 'idx': [[0, 4]]},
 {'dets': array([0.19217934], dtype=float32), 'idx': [[0, 5]]},
 {'dets': array([0.9179947], dtype=float32), 'idx': [[0, 6]]},
 {'dets': array([0.35610446], dtype=float32), 'idx': [[0, 7]]},
 {'dets': array([0.6985991], dtype=float32), 'idx': [[0, 8]]},
 {'dets': array([0.7818428], dtype=float32), 'idx': [[0, 9]]},
 {'dets': array([0.8366854], dtype=float32), 'idx': [[1, 2]]},
 {'dets': array([0.92447656], dtype=float32), 'idx': [[1, 3]]},
 {'dets': array([0.913368], dtype=float32), 'idx': [[1, 4]]},
 {'dets': array([0.6739752], dtype=float32), 'idx': [[1, 5]]},
 {'dets': array([0.68964756], dtype=float32), 'idx': [[1, 6]]},
 {'dets': array([0.36768046], dtype=float32), 'idx': [[1, 7]]},
 {'dets': array([0.46805337], dtype=float32), 'i

In [27]:
responses[9]

[{'dets': array([0.00193052], dtype=float32),
  'idx': [[0, 1, 2, 3, 4, 5, 6, 7, 8]]},
 {'dets': array([7.544615e-05], dtype=float32),
  'idx': [[0, 1, 2, 3, 4, 5, 6, 7, 9]]},
 {'dets': array([0.00207499], dtype=float32),
  'idx': [[0, 1, 2, 3, 4, 5, 6, 8, 9]]},
 {'dets': array([0.00151832], dtype=float32),
  'idx': [[0, 1, 2, 3, 4, 5, 7, 8, 9]]},
 {'dets': array([0.0075728], dtype=float32),
  'idx': [[0, 1, 2, 3, 4, 6, 7, 8, 9]]},
 {'dets': array([0.00160432], dtype=float32),
  'idx': [[0, 1, 2, 3, 5, 6, 7, 8, 9]]},
 {'dets': array([0.00038815], dtype=float32),
  'idx': [[0, 1, 2, 4, 5, 6, 7, 8, 9]]},
 {'dets': array([0.00281751], dtype=float32),
  'idx': [[0, 1, 3, 4, 5, 6, 7, 8, 9]]},
 {'dets': array([0.00222072], dtype=float32),
  'idx': [[0, 2, 3, 4, 5, 6, 7, 8, 9]]},
 {'dets': array([0.0031944], dtype=float32),
  'idx': [[1, 2, 3, 4, 5, 6, 7, 8, 9]]},
 {'dets': [], 'idx': [], 'matrices': []},
 {'dets': [], 'idx': [], 'matrices': []},
 {'dets': [], 'idx': [], 'matrices': []},
 {'d

In [22]:
next(indices_scatter[2][6])

(0, 7)

In [23]:
idxs = []

In [24]:
for i in range (2,3):
    for el in indices_scatter[i]:
        for subel in el:
            idxs.append(subel)

In [19]:
idxs

[(0, 1), (0, 3), (0, 6), (1, 2), (1, 7), (2, 6), (3, 7), (5, 6), (8, 9)]

In [65]:
det_calcular[2]

(45, 1)

In [68]:
a = indices_scatter[2]

In [69]:
len(a)

45

Exception in callback BaseAsyncIOLoop._handle_events(48, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(48, 1)>
Traceback (most recent call last):
  File "/opt/miniconda/envs/jupyterhub/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/opt/miniconda/envs/jupyterhub/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 122, in _handle_events
    handler_func(fileobj, events)
  File "/opt/miniconda/envs/jupyterhub/lib/python3.6/site-packages/tornado/stack_context.py", line 300, in null_wrapper
    return fn(*args, **kwargs)
  File "/opt/miniconda/envs/jupyterhub/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 459, in _handle_events
    self._rebuild_io_state()
  File "/opt/miniconda/envs/jupyterhub/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 516, in _rebuild_io_state
    self._update_handler(state)
  File "/opt/miniconda/envs/jupyterhub/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 533,

In [56]:
b = [list(e) for e in a]

In [ ]:
b